# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [28]:
# Bibliotecas externas de manipulação e visualização dos dados
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Biblioteca para estatísticas 
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [29]:
df = pd.read_csv('./input/previsao_de_renda.csv')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

### 1°

In [31]:
lista_de_variaveis = df.columns.to_list()
lista_de_variaveis = lista_de_variaveis[3:]

In [32]:
dicionario_de_variaveis = {}
dicionario_de_variaveis['sexo'] = {df['sexo'].unique().tolist()[0]:10119}

In [33]:
dicionario_de_variaveis = {}
dicionario_de_variaveis_max = {}

for variavel in lista_de_variaveis:
    if(variavel in ['renda', 'idade', 'tempo_emprego']):
        continue
    contagem = df[variavel].value_counts()
    dicionario_de_variaveis[variavel] = pd.DataFrame({
        ('','index'): contagem.index,
        ('','valor'): contagem.values
    })
    
    dicionario_de_variaveis_max[variavel] = dicionario_de_variaveis[variavel][dicionario_de_variaveis[variavel]['','valor'] == dicionario_de_variaveis[variavel]['','valor'].max()]

dataframe_de_contagem = pd.concat(dicionario_de_variaveis, axis=1)
dataframe_de_contagem_max = pd.concat(dicionario_de_variaveis_max, axis=1)
dataframe_de_contagem_max

sexo        posse_de_veiculo       posse_de_imovel        qtd_filhos  \
                                                                          
  index  valor            index valor           index  valor      index   
0     F  10119            False  9140            True  10143          0   

           tipo_renda          educacao       estado_civil         \
                                                                    
   valor        index valor       index valor        index  valor   
0  10376  Assalariado  7633  Secundário  8895       Casado  10534   

  tipo_residencia        qt_pessoas_residencia        
                                                      
            index  valor                 index valor  
0            Casa  13532                   2.0  8181

In [34]:
lista_de_variaveis = 'np.log(renda) ~ C(sexo, Treatment("F")) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + C(qtd_filhos, Treatment(0)) + C(tipo_renda, Treatment("Assalariado")) + C(educacao, Treatment("Secundário")) + C(estado_civil, Treatment("Casado")) + C(tipo_residencia, Treatment("Casa")) + idade + tempo_emprego'

In [35]:
y, x = patsy.dmatrices(
    lista_de_variaveis,
    df,
)
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     238.2
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        22:09:37   Log-Likelihood:                -13564.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12397   BIC:                         2.741e+04
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1788      0.039    186.307      0.000       7.103       7.254
C(sexo, Treatment("F"))[T.M]                                    0.7878      0.015     53.696      0.000       0.759       0.817
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0432      0.014      3.052      0.002       0.015       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0833      0.014     -5.952      0.000      -0.111      -0.056
C(qtd_filhos, Treatment(0))[T.1]                                0.0429      0.016      2.621      0.009       0.011       0.075
C(qtd_filhos, Treatment(0))[T.2]                                0.0775      0.022      3.493      0.000       0.034       0.121
C(qtd_filhos, Treatment(0))[T.3]                               -0.0350      0.054     -0.644      0.520      -0.142       0.072
C(qtd_filhos, Treatment(0))[T.4]                                0.3678      0.175      2.096      0.036       0.024       0.712
C(qtd_filhos, Treatment(0))[T.5]                                0.4522      0.513      0.882      0.378      -0.553       1.457
C(qtd_filhos, Treatment(0))[T.7]                                0.0424      0.323      0.131      0.896      -0.591       0.676
C(qtd_filhos, Treatment(0))[T.14]                              -0.6452      0.511     -1.262      0.207      -1.647       0.357
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2240      0.241      0.929      0.353      -0.249       0.697
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1546      0.015     10.350      0.000       0.125       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3127      0.241     -1.296      0.195      -0.786       0.160
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0558      0.022      2.511      0.012       0.012       0.099
C(educacao, Treatment("Secundário"))[T.Primário]                0.0139      0.072      0.193      0.847      -0.127       0.155
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1132      0.142      0.795      0.427      -0.166       0.392
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1067      0.014      7.668      0.000       0.079       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0285      0.032     -0.882      0.378      -0.092       0.035
C(estado_civil, Treatment("Casado"))[T.Separado]                0.0349      0.028      1.242      0.214      -0.020       0.090
C(estado_civil, Treatment("Casado"))[T.Solteiro]               -0.0203      0.021     -0.991      0.322      -0.061       0.020
C(estado_ci

### 2°

In [36]:
contagem_de_variaveis_unicas = {}
contagem_de_itens_ireelevantes_por_variavel = [3, 2, 3, 4, 5]

contagem_de_variaveis_unicas["qtd_filhos"] = dataframe_de_contagem[
    "qtd_filhos", "", "index"
].nunique()
contagem_de_variaveis_unicas["tipo_renda"] = dataframe_de_contagem[
    "tipo_renda", "", "index"
].nunique()
contagem_de_variaveis_unicas["educacao"] = dataframe_de_contagem[
    "educacao", "", "index"
].nunique()
contagem_de_variaveis_unicas["estado_civil"] = dataframe_de_contagem[
    "estado_civil", "", "index"
].nunique()
contagem_de_variaveis_unicas["tipo_residencia"] = dataframe_de_contagem[
    "tipo_residencia", "", "index"
].nunique()

contagem_de_variaveis_unicas

{'qtd_filhos': 8,
 'tipo_renda': 5,
 'educacao': 5,
 'estado_civil': 5,
 'tipo_residencia': 6}

In [37]:
porcentagem_de_irrelevancia_das_variaveis = {}

for i, variavel in enumerate(contagem_de_variaveis_unicas):
    porcentagem_de_irrelevancia_das_variaveis[variavel] = round((contagem_de_itens_ireelevantes_por_variavel[i] / contagem_de_variaveis_unicas[variavel])*100, 2)

porcentagem_de_irrelevancia_das_variaveis

{'qtd_filhos': 37.5,
 'tipo_renda': 40.0,
 'educacao': 60.0,
 'estado_civil': 80.0,
 'tipo_residencia': 83.33}

In [38]:
lista_tratada = [explicativo.strip() for explicativo in lista_de_variaveis.split(sep='+')]

remocao = 'C(tipo_residencia, Treatment("Casa"))'

lista_tratada.remove(remocao)

lista_tratada = '+'.join(lista_tratada)

In [39]:
lista_tratada

'np.log(renda) ~ C(sexo, Treatment("F"))+C(posse_de_veiculo, Treatment(False))+C(posse_de_imovel, Treatment(True))+C(qtd_filhos, Treatment(0))+C(tipo_renda, Treatment("Assalariado"))+C(educacao, Treatment("Secundário"))+C(estado_civil, Treatment("Casado"))+idade+tempo_emprego'

In [40]:
y, x = patsy.dmatrices(
    lista_tratada, df
)
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.358
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.7
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        22:09:37   Log-Likelihood:                -13566.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1820      0.038    188.378      0.000       7.107       7.257
C(sexo, Treatment("F"))[T.M]                                    0.7897      0.015     53.971      0.000       0.761       0.818
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0432      0.014      3.056      0.002       0.015       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0822      0.014     -5.982      0.000      -0.109      -0.055
C(qtd_filhos, Treatment(0))[T.1]                                0.0437      0.016      2.671      0.008       0.012       0.076
C(qtd_filhos, Treatment(0))[T.2]                                0.0776      0.022      3.498      0.000       0.034       0.121
C(qtd_filhos, Treatment(0))[T.3]                               -0.0362      0.054     -0.666      0.505      -0.143       0.070
C(qtd_filhos, Treatment(0))[T.4]                                0.3663      0.175      2.088      0.037       0.022       0.710
C(qtd_filhos, Treatment(0))[T.5]                                0.4121      0.511      0.807      0.420      -0.589       1.413
C(qtd_filhos, Treatment(0))[T.7]                                0.0410      0.323      0.127      0.899      -0.593       0.675
C(qtd_filhos, Treatment(0))[T.14]                              -0.6474      0.511     -1.266      0.205      -1.650       0.355
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2251      0.241      0.933      0.351      -0.248       0.698
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1550      0.015     10.393      0.000       0.126       0.184
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3155      0.241     -1.308      0.191      -0.788       0.157
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0571      0.022      2.569      0.010       0.014       0.101
C(educacao, Treatment("Secundário"))[T.Primário]                0.0095      0.072      0.132      0.895      -0.131       0.150
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1115      0.142      0.784      0.433      -0.167       0.390
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1064      0.014      7.662      0.000       0.079       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0282      0.032     -0.875      0.381      -0.091       0.035
C(estado_civil, Treatment("Casado"))[T.Separado]                0.0359      0.028      1.276      0.202      -0.019       0.091
C(estado_civil, Treatment("Casado"))[T.Solteiro]               -0.0189      0.020     -0.923      0.356      -0.059       0.021
C(estado_ci

Resposta: Retirando apenas a variável 'tipo_residencia' que é a considerada mais irrelevante, por ter um número superior de categorias demosntrando um p-value fora dos padrões não houveram grandes mudanças no R-quadrado e R-quadrado ajustado, ambos mantiveram os mesmos valores, já o índice AIC apresentou uma queda nos seus valores, então em última instância houve sim uma melhora do modelo mas não foi algo de grande expressão.

### 3°

In [41]:
p_values = pd.Series(reg.pvalues, index= x.design_info.column_names)
variaveis_significativas = p_values[p_values <= 0.05].index.to_list()
print(variaveis_significativas)

['Intercept', 'C(sexo, Treatment("F"))[T.M]', 'C(posse_de_veiculo, Treatment(False))[T.True]', 'C(posse_de_imovel, Treatment(True))[T.False]', 'C(qtd_filhos, Treatment(0))[T.1]', 'C(qtd_filhos, Treatment(0))[T.2]', 'C(qtd_filhos, Treatment(0))[T.4]', 'C(tipo_renda, Treatment("Assalariado"))[T.Empresário]', 'C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]', 'C(educacao, Treatment("Secundário"))[T.Superior completo]', 'idade', 'tempo_emprego']


In [42]:
y, x = patsy.dmatrices(
    'np.log(renda) ~ C(sexo, Treatment("F")) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True)) + C(tipo_renda, Treatment("Assalariado")) + C(qtd_filhos, Treatment(0)) + C(educacao, Treatment("Secundário")) + idade + tempo_emprego',
    df,
)
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     344.7
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        22:09:38   Log-Likelihood:                -13570.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12406   BIC:                         2.734e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================================================================
                                                                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------
Intercept                                                       7.1650      0.037    193.870      0.000       7.093       7.237
C(sexo, Treatment("F"))[T.M]                                    0.7872      0.015     54.123      0.000       0.759       0.816
C(posse_de_veiculo, Treatment(False))[T.True]                   0.0436      0.014      3.105      0.002       0.016       0.071
C(posse_de_imovel, Treatment(True))[T.False]                   -0.0818      0.014     -5.960      0.000      -0.109      -0.055
C(tipo_renda, Treatment("Assalariado"))[T.Bolsista]             0.2212      0.241      0.917      0.359      -0.251       0.694
C(tipo_renda, Treatment("Assalariado"))[T.Empresário]           0.1533      0.015     10.292      0.000       0.124       0.182
C(tipo_renda, Treatment("Assalariado"))[T.Pensionista]         -0.3273      0.241     -1.358      0.174      -0.800       0.145
C(tipo_renda, Treatment("Assalariado"))[T.Servidor público]     0.0570      0.022      2.567      0.010       0.013       0.100
C(qtd_filhos, Treatment(0))[T.1]                                0.0462      0.016      2.853      0.004       0.014       0.078
C(qtd_filhos, Treatment(0))[T.2]                                0.0796      0.022      3.632      0.000       0.037       0.122
C(qtd_filhos, Treatment(0))[T.3]                               -0.0345      0.054     -0.636      0.525      -0.141       0.072
C(qtd_filhos, Treatment(0))[T.4]                                0.3691      0.175      2.104      0.035       0.025       0.713
C(qtd_filhos, Treatment(0))[T.5]                                0.4330      0.511      0.848      0.396      -0.568       1.434
C(qtd_filhos, Treatment(0))[T.7]                                0.0423      0.323      0.131      0.896      -0.591       0.676
C(qtd_filhos, Treatment(0))[T.14]                              -0.6123      0.511     -1.199      0.230      -1.613       0.389
C(educacao, Treatment("Secundário"))[T.Primário]                0.0159      0.072      0.222      0.824      -0.125       0.157
C(educacao, Treatment("Secundário"))[T.Pós graduação]           0.1060      0.142      0.745      0.456      -0.173       0.385
C(educacao, Treatment("Secundário"))[T.Superior completo]       0.1067      0.014      7.687      0.000       0.079       0.134
C(educacao, Treatment("Secundário"))[T.Superior incompleto]    -0.0303      0.032     -0.941      0.347      -0.093       0.033
idade                                                           0.0059      0.001      7.544      0.000       0.004       0.007
tempo_emprego                                                   0.0615      0.001     59.303      0.000       0.059       0.064
===========

In [43]:
y, x = patsy.dmatrices(
    'np.log(renda) ~ C(sexo, Treatment("F")) + C(posse_de_veiculo, Treatment(False)) + C(posse_de_imovel, Treatment(True))  + idade + tempo_emprego  ',
    df,
)
reg = sm.OLS(y, x).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.346
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1315.
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        22:09:38   Log-Likelihood:                -13676.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12421   BIC:                         2.741e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
Intercept                                         7.3430      0.032    230.088      0.000       7.280       7.406
C(sexo, Treatment("F"))[T.M]                      0.7700      0.015     52.851      0.000       0.741       0.799
C(posse_de_veiculo, Treatment(False))[T.True]     0.0579      0.014      4.116      0.000       0.030       0.085
C(posse_de_imovel, Treatment(True))[T.False]     -0.0880      0.014     -6.379      0.000      -0.115      -0.061
idade                                             0.0044      0.001      5.871      0.000       0.003       0.006
tempo_emprego                                     0.0611      0.001     59.199      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.248   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.536   Jarque-Bera (JB):                1.221
Skew:                           0.022   Prob(JB):                        0.543
Kurtosis:                       3.019   Cond. No.                         211.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Resposta: O melhor modelo levando em consideração as métricas de AIC e R-Quadrado, foi o formatado por último com a remoção das variáveis 'estado_civil' e 'tipo_residencia' que eram as duas que se encontravam com o maior número de componentes fora dos padrões de p-value, ao retirarmos mais variáveis como 'qtd_filhos' 'educacao' e 'tipo_renda' notamos uma queda nos indices que fazem a relação de performance com complexidade do modelo, então a última decisão foi de retirar apenas as duas variáveis mais problemáticas